### (Arquivo1) Preparação dos dados:

Realização das etapas:
- Carregar dataset
- Aplicar filtros temporais
- Realizar segmentações (tempo | frequencia)

In [1]:
# Carregamento dos datasets

import numpy as np


data1 = np.load('datasets/Gabi.npy')
data2 = np.load('datasets/Jesse.npy')

data1.shape, data2.shape
# 28 -> amostras de 4 classes (movimentos) (7 instancias de cada classe)
# 1000 -> 5segundos * 200HZ (cada uma dos movimentos em uma taxa de amostragem de 200HZ)
# 4 -> eletrodos (os dois primeiros eletrodos são os que possuem dados, exemplo: print(data1))

((28, 1000, 4), (28, 1000, 4))

In [2]:
# Correção do número de eletrodos e unificação

# Como dito anteriormente, os dois primeiros eletrodos são funcionais
data1 = data1[:,:,:2]
data2 = data2[:,:,:2]

print(data1.shape, data2.shape)

data = np.array([data1, data2])
print(data.shape)

# Colocando os dados na dimensão final
data = data.transpose(0, 1, 3, 2)
print(data.shape)

(28, 1000, 2) (28, 1000, 2)
(2, 28, 1000, 2)
(2, 28, 2, 1000)


In [3]:
# Funções para aplicação dos filtros temporais

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
from sklearn.preprocessing import LabelEncoder
import mne


# definições de filtros

def butter_bandpass(data, lowcut, highcut, fs=512, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=512, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data)

###  Tarefa 1: Realizem a alicacao dos filtros para toda a base.

In [4]:
# Aplicação dos filtros de 5 - 50HZ

# código...

In [5]:
# Juntando participantes (apenas para demonstracao)

data = data.reshape(data.shape[0] * data.shape[1], data.shape[2], data.shape[3])
data.shape

(56, 2, 1000)

In [6]:
# Aplicação da segmentação (dominio, tempo e frequencia)

from scipy.signal import stft

step = 29 # Pula a cada deslizamento
segment = 64
print('Shape inicial:', data.shape)

n_win = int((data.shape[-1] - segment) / step) + 1
ids = np.arange(n_win) * step

# Janelas do dado no dominio do tempo
chunks_time = np.array([data[:,:,k:(k + segment)] for k in ids]).transpose(1, 2, 0, 3)

# Janelas do dado no domínio da frequência
_, _, chunks_freq = stft(data, fs=200, nperseg=64, noverlap=32)
chunks_freq = np.swapaxes(chunks_freq, 2, 3)

print('Formato (shape) dos dados depois da divisão de janelas')
print(f'Dominio do tempo: {chunks_time.shape} - (classes+participantes, canais, janelas, linhas)')
print(f'Dominio da frequência:  {chunks_freq.shape} - (classes+ensaios, canais, janelas, linhas)')

Shape inicial: (56, 2, 1000)
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (56, 2, 33, 64) - (classes+participantes, canais, janelas, linhas)
Dominio da frequência:  (56, 2, 33, 33) - (classes+ensaios, canais, janelas, linhas)


### Tarefa 2: Separar os participantes e armazenar os dados em disco para a execução das próximas tarefas.